In [1]:
from langchain.prompts import PromptTemplate,SystemMessagePromptTemplate,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain,LLMChain,TransformChain

f=open('../openai_api_key.txt')
api_key=f.read()
llm=ChatOpenAI(openai_api_key=api_key)

C:\Users\PC\anaconda3\envs\langchain\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# Guardar en BD Vector

In [2]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore

In [3]:
from langchain.document_loaders import WikipediaLoader
wikiloader=WikipediaLoader(query="Lolabúm",lang='es')
documents=wikiloader.load()

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs=text_splitter.split_documents(documents)

Created a chunk of size 1159, which is longer than the specified 500
Created a chunk of size 548, which is longer than the specified 500
Created a chunk of size 501, which is longer than the specified 500


In [5]:
embedding_function=OllamaEmbeddings(model='nomic-embed-text:latest')

In [6]:
vector_store_connection=SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embedding_function,
    persist_path='../persistence/bloque3/ejemplos_embedding',
    serializer="parquet"
)
vector_store_connection.persist()

# Conexión en BD Vector

In [7]:
embedding_function=OllamaEmbeddings(model='nomic-embed-text:latest')

In [8]:
vector_store_connection=SKLearnVectorStore(
    embedding=embedding_function,
    persist_path='../persistence/bloque3/ejemplos_embedding',
    serializer="parquet"
)

# Cargar cadenas QA

In [9]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [10]:
chain=load_qa_chain(
    llm,
    chain_type='stuff'
)

C:\Users\PC\AppData\Local\Temp\ipykernel_28984\1188944448.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain=load_qa_chain(


In [23]:
question="¿En qué ciudades y años ha tocado la banda Lolabúm?"

In [24]:
docs=vector_store_connection.similarity_search(question)

In [25]:
docs

[Document(metadata={'id': '365f1cb3-2985-45fa-9db4-72160f591a2d', 'source': 'https://es.wikipedia.org/wiki/Lolab%C3%BAm', 'summary': 'Lolabúm es una banda de rock ecuatoriano formada en Quito a mediados del 2014. Está conformada actualmente por el cantante y multinstrumentista Pedro Bonfim, el bajista Martín Erazo, el guitarrista Joaquín Prado y el baterista José Miguel Fabre.  La banda entró a la escena musical ecuatoriana en 2015 con varios conciertos en diferentes ciudades de Ecuador. En 2016 lanzaron su primer álbum de estudio El Cielo con el cual exploraron un sonido de rock y rock alternativo con una producción similar al lo-fi. En julio de 2018, la banda sacó su segundo álbum Tristes Trópicos, con letras de mayor profundidad y sonidos diversos tomando elementos de géneros como el reguetón, la música latinoamericana y el hip-hop. La banda forma parte de la nueva ola de artistas ecuatorianos la cual se ha desarrollado durante la década del 2010. \nActualmente se localiza en Colomb

In [26]:
chain.run(input_documents=docs,question=question)

'La banda Lolabúm ha tocado en diferentes ciudades de Ecuador desde su ingreso a la escena musical en 2015, como: Quito, Ibarra, y también ha sido teloneros en conciertos como el de la banda colombiana Monsieur Periné. No se proporciona una lista exhaustiva de todas las ciudades y años en los que han tocado.'

# Alternativa con método invoke

In [27]:
input={
    "input_documents":docs,
    "question":question
}

In [28]:
chain.invoke(input)['output_text']

'Lolabúm ha tocado en diferentes ciudades de Ecuador a lo largo de su carrera musical, incluyendo conciertos en Ibarra y en conciertos como teloneros de bandas como Monsieur Periné. Sin embargo, el texto no especifica las ciudades exactas ni los años específicos en los que la banda ha tocado.'